In [1]:
from efficacy_utils import *

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


<IPython.core.display.Javascript object>

In [2]:
import jsonmemo as jsonmemo_module
jsonmemo_module.set_lowmem(True)

In [5]:
installs_with_experiment_vars = get_installs_with_experiment_vars()

In [6]:
print(len(installs_with_experiment_vars))

11916


In [51]:
intervention_name_to_difficulty = {
  'facebook/remove_news_feed': 1,
  #'facebook/prompt_reason': 2,
  'facebook/remove_clickbait': 1,
  #'internal/choose_difficulty',
  #'facebook/show_user_info_banner',
  'facebook/toast_notifications': 2,
  'facebook/feed_injection_timer': 1,
  'facebook/remove_comments': 2,
  'facebook/block_after_interval_per_visit': 2,
  'facebook/show_user_info_interstitial': 3,
  'facebook/close_tab_timer': 3,
  'facebook/scroll_blocker': 3,
  'facebook/make_user_wait': 2,
  'facebook/rich_notifications': 1,
  'facebook/show_timer_banner': 1,
}

session_info_list = []

intervention_set = set()
difficulty_to_session_lengths = {}
for install in installs_with_experiment_vars: #[:100]:
  domain_to_intervention_to_session_lengths = get_domain_to_intervention_to_session_lengths_for_install(install)
  for domain,intervention_to_session_lengths in domain_to_intervention_to_session_lengths.items():
    if domain != 'www.facebook.com':
      continue
    for intervention,session_lengths in intervention_to_session_lengths.items():
      difficulty = intervention_name_to_difficulty.get(intervention)
      if difficulty == None:
        continue
      if difficulty not in difficulty_to_session_lengths:
        difficulty_to_session_lengths[difficulty] = []
      for session_length in session_lengths:
        difficulty_to_session_lengths[difficulty].append(session_length)
        session_info_list.append({
          'user': install,
          'difficulty': difficulty,
          'logtime': math.log(session_length),
        })

In [52]:
from browser_libs import convert_list_of_dicts_into_dataframe

df = convert_list_of_dicts_into_dataframe(session_info_list)

In [53]:
%Rpush df

In [54]:
%%R
library(lme4)
#library(sjPlot)
library(lmerTest)
library(stargazer)

In [55]:
%%R

df$user <- as.factor(df$user)
df$difficulty <- factor(df$difficulty, levels = c(1, 2, 3))
df$logtime <- as.numeric(df$logtime)
summary(df)

                       user         difficulty    logtime      
 a85b38dea50b24bc26604e2d:  22596   1:990002   Min.   : 0.000  
 b9029ed18ed93bd058ee0d3e:  17975   2:513790   1st Qu.: 3.871  
 d118e2dcfb05429f3dceaff4:  16698   3:131557   Median : 5.159  
 027223e0aebde2a4356b0b33:  13884              Mean   : 5.071  
 52ab95cbc4a45d2b751c65cd:  13773              3rd Qu.: 6.353  
 fe3a90a6471f28ba6b7f3608:  12791              Max.   :11.138  
 (Other)                 :1537632                              


In [56]:
%%R

results <- lmer(logtime ~ difficulty + (1|user), data = df)
#results <- lmer(logtime ~ condition + (1|user) + (1|domain), data = df)
show(results)
show(summary(results))
class(results) <- "lmerMod"
stargazer(results)

Linear mixed model fit by REML ['lmerModLmerTest']
Formula: logtime ~ difficulty + (1 | user)
   Data: df
REML criterion at convergence: 6029950
Random effects:
 Groups   Name        Std.Dev.
 user     (Intercept) 1.002   
 Residual             1.521   
Number of obs: 1635349, groups:  user, 6136
Fixed Effects:
(Intercept)  difficulty2  difficulty3  
    4.40237      0.00239     -0.18027  
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: logtime ~ difficulty + (1 | user)
   Data: df

REML criterion at convergence: 6029950

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-5.8286 -0.5836  0.1337  0.7113  3.5458 

Random effects:
 Groups   Name        Variance Std.Dev.
 user     (Intercept) 1.004    1.002   
 Residual             2.312    1.521   
Number of obs: 1635349, groups:  user, 6136

Fixed effects:
              Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)  4.402e+00  1.413e-02  5.577e+03 311.455   <2e-16

In [25]:
import numpy as np

In [57]:
difficulty_to_median_session_lengths = {}
for difficulty,session_lengths in difficulty_to_session_lengths.items():
  difficulty_to_median_session_lengths[difficulty] = np.mean(session_lengths)


In [58]:
from plot_utils import *

In [59]:
plot_dict_as_bar(difficulty_to_median_session_lengths)

In [11]:
print(intervention_set)

{'facebook/remove_news_feed', 'facebook/prompt_reason', 'facebook/remove_clickbait', 'internal/choose_difficulty', 'facebook/show_user_info_banner', 'facebook/toast_notifications', 'facebook/feed_injection_timer', 'facebook/remove_comments', 'facebook/block_after_interval_per_visit', 'facebook/show_user_info_interstitial', 'facebook/close_tab_timer', 'facebook/scroll_blocker', 'facebook/make_user_wait', 'facebook/rich_notifications', 'facebook/show_timer_banner'}
